<a href="https://colab.research.google.com/github/brunoras/nlpjuridico/blob/main/Raspagem_TJBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 30.8 MB/s 
     |████████████████████████████████| 140 kB 85.2 MB/s 
     |████████████████████████████████| 384 kB 62.0 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubunt

In [ ]:
import pandas as pd
import time
import math
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

##Criando o Driver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.get('https://jurisprudencia.tjba.jus.br/')

## Pesquisando

In [ ]:
termo = 'droga'
textos = []
classes = []

In [ ]:
element = driver.find_element(By.NAME, 'assunto')
element.clear()
element.send_keys(termo)
print('Termo da pesquisa: ' + element.get_attribute('value'))

button = driver.find_element(By.CLASS_NAME, 'btn-custom-action')
button.click()
time.sleep(3)

Termo da pesquisa: droga


In [ ]:
result_count = driver.find_elements(By.CLASS_NAME, 'text')
count = result_count[-1].text
count = count.replace(' Acórdãos encontrados', '')
pgs = math.ceil(int(count)/10)
print(str(pgs) + ' páginas de resultados')

702 páginas de resultados


In [ ]:
def raspe(classe, texto):
  # clicando nos ler mais
  lermais = driver.find_elements(By.CLASS_NAME, 'read-more-button')
  for b in lermais:
    b.click()
  
  # raspagem do texto
  scrap = driver.find_elements(By.CLASS_NAME, 'displayed-text')
  for s in scrap:
    text = s.text
    text = text.replace('\nLer menos', '')
    texto.append(text)

  # raspagem da classe
  scrap = driver.find_elements(By.CLASS_NAME, 'fieldValue')
  temp = []
  for s in scrap:
    text = s.text
    temp.append(text)
  for i in range(len(temp)):
    if i%6 == 5:
      classe.append(temp[i])

In [ ]:
def proximo():
  pages = driver.find_elements(By.CLASS_NAME, 'arrow')
  pages[-1].click()
  time.sleep(1)

## Raspagem

In [ ]:
for i in range(pgs):
  raspe(classes, textos)
  print(i+1)
  if i < (pgs - 1):
    proximo()

## Salvando

In [ ]:
result = []
for c, t in zip(classes, textos):
  result.append([c, t])

In [ ]:
with open(r'tjba_casos_'+ termo +'.txt', 'w') as fp:
  for item in result:
    fp.write("%s\n" % item[0])
    fp.write("%s\n" % item[1])
  print('Done')

Done


## Fechando o Browser

In [ ]:
driver.quit()